In [292]:
import pandas as pd
import numpy as np
import os
import warnings
from scipy import stats
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS
import math
import calendar
from matplotlib import pyplot as plt
import pickle
from itertools import combinations
import datetime
from datetime import datetime as dt
from functools import reduce
%pylab inline
warnings.filterwarnings("ignore")
os.chdir(r'''D:\Coursera_ML\Final Project\Data''')

Populating the interactive namespace from numpy and matplotlib


#### 1. Загрузка данных и новых признаков (ранее подготовленных).

In [3]:
def to_dt(x):
    return pd.datetime.strptime(x,'%Y-%m-%d %H:%M:%S')

In [4]:
grouped=pd.DataFrame()
grouped_vars=pd.DataFrame()
inregion=pd.DataFrame()
for val in ['january','february','march','april','may','june']:
    val1=pd.read_excel('Grouped_data_'+val+'.xlsx')
    grouped=pd.concat([grouped,val1])
    with open('new_vars_'+val+'.pickle','rb') as file:
        val2=pickle.load(file)
    grouped_vars=pd.concat([grouped_vars,val2])
    with open('inregion_'+val+'.pickle','rb') as file:
        val3=pickle.load(file)
    inregion=pd.concat([inregion,val3])

#### Следующие признаки были рассмотрены:
#### 1) Признаки из сырых данных - средние число пассажиров, время поездки, дальность поездки, основная сумма оплаты (fare_amount) и доплаты за часы пик и ночное время (extra) и число машин приехавших в географическую зону.
#### 2) Календарные - календарь празников США https://pypi.org/project/holidays/
#### 3) Географические - идентификатор боро
#### 4) Дополнительный признак - идентификатор аэропортов.

In [7]:
regs=grouped_vars.region_pickup.unique()
grouped=grouped[grouped.region.apply(lambda x: x in regs)]

In [8]:
date_arr=grouped.pickup_datehour.apply(to_dt).unique()

In [9]:
inregion_corrected=inregion.groupby(['dropoff_datehour','region_dropoff'])[['N_arrive']].sum().reset_index()

In [10]:
inregion_corrected['dropoff_datehour']=inregion_corrected.dropoff_datehour.apply(to_dt)

In [11]:
inregion_corrected=inregion_corrected[(inregion_corrected.dropoff_datehour<=max(date_arr))&(inregion_corrected.dropoff_datehour>=min(date_arr))]

In [12]:
inregion_corrected.columns=list(grouped_vars.columns[:2])+['N_arrive']
grouped_vars['pickup_datehour']=grouped_vars['pickup_datehour'].apply(to_dt)

In [13]:
all_vars=grouped_vars.merge(inregion_corrected,how='outer',on=list(grouped_vars.columns[:2])).fillna(0)

In [14]:
lst=list(grouped.columns)
lst[1]='region_pickup'
grouped.columns=lst
grouped['pickup_datehour']=grouped.pickup_datehour.apply(to_dt)

In [15]:
data_all=grouped.merge(all_vars,how='outer',on=list(all_vars.columns[:2])).fillna(0)
data_all.index=data_all.pickup_datehour

In [16]:
regs=data_all.region_pickup.unique()

#### 2. Посчитаем ошибку в мае без учета и с учетом новых признаков.

In [539]:
new_data={}
for val in data_all.region_pickup.unique():
    form=pd.DataFrame(0,index=date_arr,columns=list(data_all.columns[2:]))
    temp=data_all[data_all.region_pickup==val]
    form.update(temp)
    y=form[['N_trips']]
    new_data[val]=[y,form[['N_arrive','trip_distance']]]

In [41]:
exog['week_day']=[x.weekday() for x in exog.index]
exog['hour']=[x.hour for x in exog.index]

In [443]:
def outer_periods(step,p):
    def sum_periods(df):
        vals=df.values
        ar0=np.array([np.array([np.nan]*len(vals[0])) for x in range(p+step-1)])
        ar1=np.array([np.apply_along_axis(sum,0,vals[x-p-step+1:x]) for x in range(p+step-1,len(vals))])
        res=pd.DataFrame(np.vstack([ar0,ar1]),index=df.index,columns=df.columns)
        return res
    return sum_periods

In [540]:
for val in new_data:
    dat=new_data[val][0]
    shifted=pd.concat([pd.DataFrame(dat.shift(periods=i).values,\
                                    columns=dat.columns+\
                                    np.array(['_'+str(i)]*len(dat.columns)),\
                index=dat.index) for i in range(1,7)],axis=1)
    shifted_seasonal=pd.concat([pd.DataFrame(dat.shift(periods=i*24).values,\
                                    columns=dat.columns+\
                                    np.array(['_'+str(i*24)]*len(dat.columns)),\
                index=dat.index) for i in range(1,3)],axis=1)
    sum_halfday=outer_periods(1,12)(dat)
    sum_halfday.columns=[x+'_sum_12' for x in sum_halfday.columns]
    sum_day=outer_periods(1,24)(dat)
    sum_day.columns=[x+'_sum_24' for x in sum_day.columns]
    shifted_all=pd.concat([shifted,shifted_seasonal,sum_halfday,sum_day],axis=1)
    new_data[val].append(shifted_all)

In [101]:
preds_csv=pd.read_csv(r'''TRUE_PREDS_ALL.csv''',engine='python')

In [133]:
def recover_ind(x):
    date,hour,step=x.split('_')[1:]
    if len(hour)==1: hour='0'+hour
    time=hour+':00:00'
    actual=dt.strptime(date+' '+time,'%Y-%m-%d %H:%M:%S')+datetime.timedelta(hours=int(step))
    return actual

In [134]:
preds_csv.index=preds_csv.id.apply(recover_ind)

In [108]:
preds_csv['step']=preds_csv.id.apply(lambda x: int(x.split('_')[-1]))

In [109]:
preds_csv['reg']=preds_csv.id.apply(lambda x: int(x.split('_')[0]))

In [142]:
total_inds=pd.Series(np.nan,index=date_arr)
def extract_preds(reg,step):
    data=preds_csv[(preds_csv['reg']==reg)&(preds_csv['step']==step)][['y','id']]
    res=total_inds
    res[data.index]=data['y']
    return res

In [217]:
def mdot(*args):
    st=args[0]
    for val in args[1:]:
        st=np.dot(st,val)
    return st

In [114]:
for val in exog.week_day.unique()[1:]:
    exog['week_day'+str(val)]=exog.week_day.apply(lambda x: int(x==val))
for val in exog.hour.unique()[1:]:
    exog['hour'+str(val)]=exog.hour.apply(lambda x: int(x==val))

In [ ]:
exog=exog.drop(['week_day','hour'],axis=1)

In [130]:
from datetime import date
import holidays
us_holidays = holidays.UnitedStates()
def inhol(x):
    return int(x in us_holidays)

In [131]:
exog['hol']=[inhol(x) for x in exog.index]

In [ ]:
exog2=exog.drop(['hol'],axis=1)

In [388]:
len_april=np.array([x<=pd.datetime(2016,4,30,23) for x in exog.index]).argmin()
len_may=np.array([x<=pd.datetime(2016,5,31,23) for x in exog.index]).argmin()
len_june=len(exog)

In [510]:
from sklearn.linear_model import Lasso, Ridge
def outer_linreg(estimator,lambd,ind0,ind1,ind2,ind3,mode='score',old=True):
    def linreg(reg):
        y_all=new_data[reg][0]['N_trips']
        dat_all=new_data[reg][1]
        shifts=new_data[reg][2]
        res=np.array([])
        if mode=='score':
            for step in range(1,7):
                preds=extract_preds(reg,step)
                if old:
                    X_all=pd.concat([exog2,shifts,preds],axis=1)
                else: 
                    X_all=pd.concat([exog,shifts,dat_all,preds],axis=1)
                X=X_all.iloc[ind0:ind1,:].dropna(axis=0)
                y=y_all[X.index].values
                X=X.values
                model=eval(estimator)(alpha=lambd).fit(X,y)
                X_test=X_all.iloc[ind2+step-1:ind3-6+step,:].dropna(axis=0)
                y_test=y_all[X_test.index].values
                X_test=X_test.values
                prd=mdot(X_test,model.coef_)
                res=np.append(res,np.abs(y_test-prd))
            return res
        elif mode=='data':
            new_preds=pd.DataFrame()
            for step in range(1,7):
                preds=extract_preds(reg,step)
                if old:
                    X_all=pd.concat([exog2,shifts,preds],axis=1)
                else: 
                    X_all=pd.concat([exog,shifts,dat_all,preds],axis=1)
                X=X_all.iloc[ind0:ind1,:].dropna(axis=0)
                y=y_all[X.index].values
                X=X.values
                model=eval(estimator)(alpha=lambd).fit(X,y)
                X_test=X_all.iloc[ind2+step-1:ind3-6+step,:].dropna(axis=0)
                data=preds_csv[(preds_csv['reg']==reg)&(preds_csv['step']==step)].loc[list(X_test.index),['id']]
                y_test=y_all[X_test.index].values
                X_test=X_test.values
                prd=mdot(X_test,model.coef_)
                data['y']=prd
                res=np.append(res,np.abs(y_test-prd))
                new_preds=pd.concat([new_preds,data])
            return res,new_preds
    return linreg

In [519]:
f_old=outer_linreg('Lasso',1,0,len_april,len_april,len_may,mode='score',old=True)

In [520]:
lists_old=[f_old(x) for x in new_data]

In [521]:
print('Ошибка в мае с использованием старых признаков={}'.format(np.mean(lists_old)))

Ошибка в мае с использованием старых признаков=24.328145594212064


In [522]:
f_new=outer_linreg('Lasso',1,0,len_april,len_april,len_may,mode='score',old=False)

In [523]:
lists_new=[f_new(x) for x in new_data]

In [524]:
print('Ошибка в мае с добавлением новых признаков={}'.format(np.mean(lists_new)))

Ошибка в мае с добавлением новых признаков=20.634259379157808


#### Таким образом ошибка в мае снизилась с 24.3 до 20.6. Теперь посчитаем ошибку в июне и осуществим сабмишн.

In [529]:
f_new=outer_linreg('Lasso',1,0,len_may,len_may,len_june,mode='data',old=False)

In [530]:
lists_new=[f_new(x) for x in new_data]

In [535]:
err=np.mean([x[0] for x in lists_new])
data=pd.concat([x[1] for x in lists_new])

In [536]:
print('Ошибка в июне с добавлением новых признаков={}'.format(np.mean(err)))

Ошибка в июне с добавлением новых признаков=19.72713578467824


#### Ошибка в июне составила 19.7

In [538]:
data.to_csv('results_final6.csv',index=False)

# Ссылка на сабмишн: https://www.kaggle.com/c/yellowtaxi/submissions?sortBy=date&group=all&page=1